In [6]:
import keras
from keras import layers

import numpy as np
import random
import io, os, re

In [25]:
with io.open("./data/Lisova_pisnya_1371650989.txt", encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")
text = text.replace("  ", " ")
text = str_output = re.sub(r'[^а-яА-ЯіїєґІЇЄҐ\s]', "", text)

chars = sorted(list(set(text)))

char_map = dict((c, i) for i, c in enumerate(chars))
reverse_char_map = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next.append(text[i + maxlen])

print("Corpus length:", len(text))
print("Total chars:", len(chars))
print("Sequences len:", len(sentences))
print("Sentence example:")
print("\t", sentences[0])
print("\t", sentences[1])
print("\t", sentences[2])
print(char_map)
print(next[:10])

Corpus length: 82990
Total chars: 33
Sequences len: 27650
Sentence example:
	 лісова пісня українка леся  пролог  той 
	 ова пісня українка леся  пролог  той що 
	  пісня українка леся  пролог  той що гре
{' ': 0, 'а': 1, 'б': 2, 'в': 3, 'г': 4, 'д': 5, 'е': 6, 'ж': 7, 'з': 8, 'и': 9, 'й': 10, 'к': 11, 'л': 12, 'м': 13, 'н': 14, 'о': 15, 'п': 16, 'р': 17, 'с': 18, 'т': 19, 'у': 20, 'ф': 21, 'х': 22, 'ц': 23, 'ч': 24, 'ш': 25, 'щ': 26, 'ь': 27, 'ю': 28, 'я': 29, 'є': 30, 'і': 31, 'ї': 32}
['щ', 'г', 'б', ' ', 'е', ' ', 'у', 'л', ' ', 'т']


In [19]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_map[char]] = 1
    y[i, char_map[next[i]]] = 1

# x - кодування слів
print(x[0])
print(y[0])

[[False False False ... False False False]
 [False False False ... False  True False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [ True False False ... False False False]]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False]


In [26]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.05)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [32]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [33]:
epochs = 10
txt_gen_len = 400
batch_size = 128

model.fit(x, y, batch_size=batch_size, epochs=epochs)

start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text[start_index: start_index + maxlen]
    print('...Generating with seed: "' + sentence + '"')

    for i in range(txt_gen_len):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_map[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = reverse_char_map[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print("...Generated: ", generated)
    print("-")

217/217 [==============================] - 10s 44ms/step - loss: 1.5926
...Diversity: 0.2
...Generating with seed: " брали сирого дерева р у с а л к а      "
...Generated:                                                                                                                                                                                                                                                                                                                                                                                                                  
-
...Diversity: 0.5
...Generating with seed: " брали сирого дерева р у с а л к а      "


/var/folders/pf/0l4vr5sx3458dlwp6hhfl46r0000gn/T/ipykernel_44738/2351311327.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:                                                                                            не тодитько довкуває запускається в лісі вибатуться в ницволо поміває в лісі с о д я н и к у там зводати спускається як же половати так так ти з вістить л у к а ш як на так як так                    л у к а ш яканось ми                                                                                           
-
...Diversity: 1.0
...Generating with seed: " брали сирого дерева р у с а л к а      "
...Generated:  ні шебеться в ходути я більну був ти дориниць і бе тепець оз вокилина осінкий зашрускатух ход пратись то водой ти хто сяє вісів то так воне пролитно в хатчу вже ж бо л у к а ш      ти град килини п е р е м а в кую лусаахв л у к а ш які глябім налепмаває і вибиться оглядається се жнала застить люба все навасів була та було       тихоці втать висмавою на то тра      які з осжши сліниться невід галяв
-
...Diversity: 1.2
...Generating with seed: " брали сирого дерева р у с а л к а